In [1]:
from pyIClab import Eluent, Detector
from pyIClab import ContaminatedPhreeqcSuppressor as Suppressor
from pyIClab import __version__ as pyIClab__version__
import pint

pyIClab__version__

'2024.6.6.1'

In [2]:
# Don't do it in the real world
# They just interfere with each other
# feel free to replace Na2SO4 with any salts
eluent = Eluent('DirtyEluent', profile={'Na+': '2 mM', 'SO4-2': '1 mM'})
d1 = Detector('FrontDetector', freq='5 Hz')
d2 = Detector('RearDetector', freq='5 Hz')
s1 = Suppressor('AionSuppressor', kind='anion', _CO2_level=0.1)
s2 = Suppressor('CationSuppressor', kind='cation', _CO2_level=0.1)
eluent.assemble(d1)
d1.assemble(s1)
s1.assemble(s2)
s2.assemble(d2)
print(eluent.repr_flow_line())

<Eluent "DirtyEluent" Isocratic(Na[+1]: 2.0 mM, SO4[-2]: 1.0 mM) in 10 min> -> <Detector "FrontDetector"> -> <Suppressor "AionSuppressor"> -> <Suppressor "CationSuppressor"> -> <Detector "RearDetector"> -> <Waste>
Flow Rate: 1.0 mL/min


In [3]:
eluent._cur_time = pint.Quantity('2.0 min')
for accessory in eluent.flow_line[1:]:
    accessory.flush()

Suppressing eluent on <Suppressor "AionSuppressor">...:   0%|          | 0/8000 [00:00<?, ?it/s]

Suppressing eluent on <Suppressor "CationSuppressor">...:   0%|          | 0/8000 [00:00<?, ?it/s]

In [4]:
x, y = d1.get_signals(signal_type='conductivity', tmax=0.1).to_numpy().T
print(f'Conductivity of Effluent from {d1}: {y[0]:.3f} μS/cm')
print(f'Compoments: {", ".join(d1(0).keys())}')

Calculating eluent conductivity on <Detector "FrontDetector">...:   0%|          | 0/30 [00:00<?, ?it/s]

Conductivity of Effluent from <Detector "FrontDetector">: 268.627 μS/cm
Compoments: Na[+1], SO4[-2]


In [5]:
x, y = d2.get_signals(signal_type='conductivity', tmax=0.1).to_numpy().T
print(f'Conductivity from Effluent of {d2}: {y[0]:.3f} μS/cm')
print(f'Compoments: {", ".join(d2(0).keys())}')

Calculating eluent conductivity on <Detector "RearDetector">...:   0%|          | 0/30 [00:00<?, ?it/s]

Conductivity from Effluent of <Detector "RearDetector">: 4.160 μS/cm
Compoments: H[+1], CO2, OH[-1], HCO3[-1], CO3[-2]
